In [30]:
import geemap
import ee
geemap.ee.Initialize()

In [31]:
Map = geemap.Map()
# Map

In [32]:
aoi = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME','New Zealand')).geometry()


Map.centerObject(aoi,10)
Map

Map(center=[-41.69398314807406, 172.87124491464206], controls=(WidgetControl(options=['position', 'transparent…

In [33]:
# Load the CHIRPS precipitation dataset
chirps = ee.ImageCollection('UCSB-CHG/CHIRPS/PENTAD')

year = 2022
startDate = ee.Date.fromYMD(year, 1, 1)
endDate = startDate.advance(1, 'year')
filtered = chirps.filter(ee.Filter.date(startDate, endDate))

total = filtered.reduce(ee.Reducer.sum())

palette = ['#ffffcc','#a1dab4','#41b6c4','#2c7fb8','#253494']
visParams = {
    'min': 0,
    'max': 2000,
    'palette': palette
}
Map.addLayer(total, visParams, 'Total Precipitation')

stats = total.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
)
print(stats.get('precipitation_sum'))

# Clip the image to the city boundary and display
totalaoi = total.clip(aoi)
Map.addLayer(totalaoi, visParams, 'New Zealand')

# Calculate average rainfall for past 35+ years
years = ee.List.sequence(1981, 2019)
print(years)

# def yearlyRainfall(year):
#     startDate = ee.Date.fromYMD(year, 1, 1)
#     endDate = startDate.advance(1, 'year')
#     filtered = chirps.filter(ee.Filter.date(startDate, endDate))
#     total = filtered.reduce(ee.Reducer.sum())
#     stats = total.reduceRegion(
#         reducer=ee.Reducer.mean(),
#         geometry=aoi,
#         scale=30,
#         maxPixels=1e9
#     )
#     f = ee.Feature(None, {
#         'year': year,
#         'precipitation': stats.get('precipitation_sum')
#     })
#     return f

# rainfallYears = ee.FeatureCollection(
#   years.map(yearlyRainfall))

# Display the map
Map.addLayer(aoi, {}, 'New Zealand')
Map.centerObject(aoi, 6)
Map


ee.ComputedObject({
  "functionInvocationValue": {
    "functionName": "Dictionary.get",
    "arguments": {
      "dictionary": {
        "functionInvocationValue": {
          "functionName": "Image.reduceRegion",
          "arguments": {
            "geometry": {
              "functionInvocationValue": {
                "functionName": "Collection.geometry",
                "arguments": {
                  "collection": {
                    "functionInvocationValue": {
                      "functionName": "Collection.filter",
                      "arguments": {
                        "collection": {
                          "functionInvocationValue": {
                            "functionName": "Collection.loadTable",
                            "arguments": {
                              "tableId": {
                                "constantValue": "FAO/GAUL/2015/level0"
                              }
                            }
                          }
               

Map(bottom=164833.0, center=[-41.69398314807406, 172.87124491464206], controls=(WidgetControl(options=['positi…

In [ ]:
print(stats.get('Collection.geometry'))

In [ ]:
# task = ee.batch.Export.table.toDrive(
# collection=ee.FeatureCollection(aoi),
# description= 'rain',
# folder='GEE_exports',
# fileFormat='CSV'
# )
# task.start()